In [2]:
%matplotlib inline

In [ ]:
# https://github.com/nrg-projects/sara-on-off-boari/blob/main/classification_kinematics.ipynb

In [7]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import LeaveOneGroupOut, HalvingGridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from IPython.display import display
from numpy import ravel

### Load csv (output from load kinematics)

In [11]:
df = pd.read_csv('output_kinematics_dataframe.csv',index_col=0).convert_dtypes()
df.head(20)

,subject_number,state,trial_number,trunk_lateral_flexion,trunk_rotation,trunk_flx_extension,left_shoulder_add_abduction,left_shoulder_int_external_rotation,left_shoulder_flx_extension,right_shoulder_add_abduction,...,right_steps_per_minute_mean,right_stride_length_mean,right_strides_per_minute_mean,speed,stride_length_mean,stride_width_mean,cycle_time_mean,double_limb_support_time_ave,right_initial_double_limb_support_time_mean,right_terminal_double_limb_support_time_mean
0,22,on,15,2.97897,7.90822,3.46405,2.62925,7.44,3.67538,2.25357,...,111.111107,1.075387,54.216866,0.953966,1.055722,0.043635,1.106667,0.276667,0.14,0.136667
1,22,on,12,2.95278,6.07138,3.67487,3.08497,5.54828,4.4002,2.24671,...,114.65432,1.024454,54.723885,0.930965,1.024061,0.040329,1.1,0.273333,0.146667,0.126667
2,22,on,11,2.86238,7.05173,2.70137,2.72333,5.98249,4.07167,2.71655,...,108.433739,1.02052,53.101505,0.901366,1.015539,0.043939,1.126667,0.28,0.146667,0.133333
3,22,on,16,2.51971,6.33676,4.21162,3.5862,8.06377,6.83211,5.36561,...,115.384636,1.051218,55.05051,0.96528,1.055373,0.056318,1.093333,0.273333,0.14,0.133333
4,22,on,10,2.13155,6.12791,2.98573,3.33085,7.23728,4.59507,1.71036,...,111.840065,1.003775,54.545452,0.907739,1.004565,0.054027,1.106667,0.294444,0.151111,0.143333
5,22,on,13,2.81942,6.70328,2.5504,2.43887,5.2067,2.91801,2.6017,...,116.883102,1.063178,56.603771,0.990234,1.052948,0.051866,1.063333,0.263333,0.136667,0.126667
6,22,on,14,2.29745,6.3868,3.12816,3.31756,8.24121,4.96836,2.90033,...,114.691544,1.066811,55.389332,0.987408,1.0664,0.043348,1.08,0.271111,0.137778,0.133333
7,22,on,5,2.56575,6.13994,3.46533,3.63583,7.22235,5.28541,3.39935,...,113.212029,1.044081,54.216873,0.930171,1.031456,0.054255,1.108889,0.278889,0.143333,0.135556
8,22,on,2,2.23096,5.40391,2.75989,4.79339,12.60892,9.92368,3.80818,...,105.897003,1.039601,52.631573,0.922987,1.044001,0.039526,1.131111,0.265556,0.123333,0.142222
9,22,on,8,2.74622,7.04495,3.09662,3.19618,6.57645,3.67301,2.82616,...,113.924057,1.082253,54.54546,0.960858,1.053741,0.042514,1.096667,0.283333,0.153333,0.13


### Missing values


#### - Identify the indexes with missing values
#### - Print the subject number, state and trial of this missing data
#### - Drop these rows and reset indexes


In [10]:
missing_rows = df.isnull().sum(axis=1)
missing_rows_filtered = missing_rows[missing_rows != 0]
print(missing_rows_filtered.index)

Index([ 19,  21,  22,  23,  24,  25,  26,  30,  31,  32,  33,  34, 231, 233,
       239, 335, 343, 349, 449, 668, 673, 674, 678, 680, 682, 685, 687, 768,
       770, 778, 779, 882, 884, 885],
      dtype='int64')


In [14]:
import pandas as pd
import pathlib as path
import numpy as np
import warnings
import re
import xlrd

In [17]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [39]:
root_folder = path.Path('C3Dfiles')

In [41]:
angles_header = ['Trunk_Lateral_Flexion', 'Trunk_Rotation', 'Trunk_Flx/Extension',
                'Left_Shoulder_Add/Abduction', 'Left_Shoulder_Int/External_Rotation', 'Left_Shoulder_Flx/Extension',
                'Right_Shoulder_Add/Abduction', 'Right_Shoulder_Int/External_Rotation', 'Right_Shoulder_Flx/Extension',
                'Left_Elbow_Add/Abduction', 'Left_Elbow_Pron/Supination', 'Left_Elbow_Flx/Extension',
                'Right_Elbow_Add/Abduction', 'Right_Elbow_Pron/Supination', 'Right_Elbow_Flx/Extension',
                'Left_Pelvic_Obliquity', 'Left_Pelvic_Rotation', 'Left_Pelvic_Tilt',
                'Right_Pelvic_Obliquity', 'Right_Pelvic_Rotation', 'Right_Pelvic_Tilt',
                'Left_Hip_Add/Abduction', 'Left_Hip_Int/External_Rotation', 'Left_Hip_Flexion/Extension',
                'Right_Hip_Add/Abduction', 'Right_Hip_Int/External_Rotation', 'Right_Hip_Flexion/Extension',
                'Left_Knee_Add/Abduction', 'Left_Knee_Int/External_Rotation', 'Left_Knee_Flx/Extension',
                'Right_Knee_Add/Abduction', 'Right_Knee_Int/External_Rotation', 'Right_Knee_Flx/Extension',
                'Left_Ankle_Inv/Eversion', 'Left_Ankle_Add/Abduction', 'Left_Ankle_Dorsi/Plantarflexion',
                'Right_Ankle_Inv/Eversion', 'Right_Ankle_Add/Abduction', 'Right_Ankle_Dorsi/Plantarflexion',
                'Left_Foot_Inv/Eversion', 'Left_Foot_Int/External_Rotation', 'Left_Foot_DF/Plantarflexion',
                'Right_Foot_Inv/Eversion', 'Right_Foot_Int/External_Rotation', 'Right_Foot_DF/Plantarflexion']

# Replace spaces and slashes with underscores in angles_header
angles_header = [header.replace(' ', '_').replace('/', '_') for header in angles_header]

In [42]:
trajectories_header = ['CoM_AP', 'CoM_Vertical', 'CoM_ML', 'CLAV_AP', 'CLAV_Vertical', 'CLAV_ML', 'STRN_AP', 
                       'STRN_Vertical', 'STRN_ML', 'C7_AP', 'C7_Vertical', 'C7_ML', 'T10_AP', 'T10_Vertical',
                       'T10_ML', 'RSHO_AP', 'RSHO_Vertical', 'RSHO_ML', 'LSHO_AP', 'LSHO_Vertical', 'LSHO_ML',
                       'RUPA_AP', 'RUPA_Vertical', 'RUPA_ML', 'REL_AP', 'REL_Vertical', 'REL_ML', 'REM_AP',
                       'REM_Vertical', 'REM_ML', 'RFRA_AP', 'RFRA_Vertical', 'RFRA_ML', 'RWL_AP', 'RWL_Vertical',
                       'RWL_ML', 'RWM_AP', 'RWM_Vertical', 'RWM_ML', 'LUPA_AP', 'LUPA_Vertical', 'LUPA_ML', 'LEL_AP',
                       'LEL_Vertical', 'LEL_ML', 'LEM_AP', 'LEM_Vertical', 'LEM_ML', 'LFRA_AP', 'LFRA_Vertical',
                       'LFRA_ML', 'LWL_AP', 'LWL_Vertical', 'LWL_ML', 'LWM_AP', 'LWM_Vertical', 'LWM_ML', 'R.ASIS_AP',
                       'R.ASIS_Vertical', 'R.ASIS_ML', 'L.ASIS_AP', 'L.ASIS_Vertical', 'L.ASIS_ML', 'R.PSIS_AP',
                       'R.PSIS_Vertical', 'R.PSIS_ML', 'L.PSIS_AP', 'L.PSIS_Vertical', 'L.PSIS_ML', 'R.GTR_AP',
                       'R.GTR_Vertical', 'R.GTR_ML', 'R.Knee_AP', 'R.Knee_Vertical', 'R.Knee_ML', 'R.HF_AP', 
                       'R.HF_Vertical', 'R.HF_ML', 'R.TT_AP', 'R.TT_Vertical', 'R.TT_ML', 'R.Ankle_AP',
                       'R.Ankle_Vertical', 'R.Ankle_ML', 'R.Heel_AP', 'R.Heel_Vertical', 'R.Heel_ML', 'R.MT1_AP',
                       'R.MT1_Vertical', 'R.MT1_ML', 'R.MT5_AP', 'R.MT5_Vertical', 'R.MT5_ML', 'L.GTR_AP',
                       'L.GTR_Vertical', 'L.GTR_ML', 'L.Knee_AP', 'L.Knee_Vertical', 'L.Knee_ML', 'L.HF_AP',
                       'L.HF_Vertical', 'L.HF_ML', 'L.TT_AP', 'L.TT_Vertical', 'L.TT_ML', 'L.Ankle_AP', 
                       'L.Ankle_Vertical', 'L.Ankle_ML', 'L.Heel_AP', 'L.Heel_Vertical', 'L.Heel_ML', 'L.MT1_AP',
                       'L.MT1_Vertical', 'L.MT1_ML', 'L.MT5_AP', 'L.MT5_Vertical', 'L.MT5_ML', 'R.Knee.Medial_AP',
                       'R.Knee.Medial_Vertical', 'R.Knee.Medial_ML', 'R.Ankle.Medial_AP', 'R.Ankle.Medial_Vertical',
                       'R.Ankle.Medial_ML', 'R.MT2_AP', 'R.MT2_Vertical', 'R.MT2_ML', 'L.Knee.Medial_AP',
                       'L.Knee.Medial_Vertical', 'L.Knee.Medial_ML', 'L.Ankle.Medial_AP', 'L.Ankle.Medial_Vertical',
                       'L.Ankle.Medial_ML', 'L.MT2_AP', 'L.MT2_Vertical', 'L.MT2_ML']

# Replace spaces and slashes with underscores in trajectories_header
trajectories_header = [trajectory.replace(' ', '_').replace('/', '_') for trajectory in trajectories_header]

In [43]:
df_angles = pd.DataFrame()
df_spatiotemporal = pd.DataFrame()
df_trajectories = pd.DataFrame()

In [79]:
# Load all folders
for subject in root_folder.glob('**/*'):
    
    #string with subject number
    subject_str = str(subject)

    # ignore files, consider just directories
    if subject.is_dir() and "SUB" in subject_str:
        
        # save strings of subject number and condition and load this data as columns of a new dataframe
        # this dataframe will not be relevant, itself, is just used as an organizing tool
        subject_number = int(subject_str[subject_str.find("SUB") + 3 : subject_str.find("SUB") + 5])
        condition = [subject_str[subject_str.find("SUB") + 6 :]]
        df_num_state = pd.DataFrame({'subject_number':subject_number, 'state':condition})
       

        # consider all (but only) csv_angles in the subject directory
        for csv_angles in subject.glob('**/*_angular_kinematics.csv'):
            
           # just like before with the subject number, we save the number of the trial in another dataframe
           trial_str = str(csv_angles)
           trial_number = int(trial_str[trial_str.find("_walk_") + 6 : trial_str.find("_angular_kinematics")])
           df_trial_num = pd.Series(trial_number, name='trial_number')

           #read csv_angles file and delete three first rows (missing info) and the fourth row (units)
           csv_ang = pd.read_excel(csv_angles, engine='xlrd')
           csv_ang.drop([0, 1, 2, 3, 4], inplace=True)

           #delete time and frame columns
           csv_ang.drop(csv_ang.columns[[0, 1]], axis = 1, inplace = True)

           #set first row as dataframe header
           #set all headers to lower letters and replace spaces and slashes with "_"
           csv_ang = csv_ang[0:]
           csv_ang.columns = angles_header
           csv_ang.columns = csv_ang.columns.str.lower()
           csv_ang.reset_index(drop=True, inplace=True)

           #calculate range of motion (ROM) for each joint and set values as a df
           #the ROM is calculated as the maximum registered angle minus the minimum one
           rom = csv_ang.max() - csv_ang.min()
           df_rom = pd.DataFrame([rom.values], columns=csv_ang.columns)

           # concatenate the three created databases: subject number, state and ROMS
           # the name of this database is df_angles
           df_trial = pd.concat([df_num_state, df_trial_num, df_rom], axis=1)
           df_angles = pd.concat([df_angles, df_trial], ignore_index=True)


        # Repeat process for spatiotemporal parameters
        for csv_spatiotemporal in subject.glob('**/*_temporal_distance.txt'):

            # just like before we save the number of the trial in another dataframe
            trial_str = str(csv_spatiotemporal)
            trial_number = int(trial_str[trial_str.find("_walk_") + 6 : trial_str.find("_temporal_distance")])
            df_trial_num = pd.Series(trial_number, name='trial_number')

            # read the trajectories csv seting the correct separator. 
            # Drop unuseful columns. Fix format. Reset index after. 
            csv_spat = pd.read_csv(csv_spatiotemporal, sep = '\t')
            csv_spat.drop([1, 2, 3], inplace=True)
            csv_spat.drop(csv_spat.columns[[0]], axis = 1, inplace = True)
            csv_spat.reset_index(drop=True, inplace=True)
            csv_spat.columns = csv_spat.iloc[0].str.lower()
            csv_spat = csv_spat[1:].reset_index(drop=True)

            # concatenate the three created databases: subject number, state, trial and spatiotemporal parameters
            # the name of this database is df_spatiotemporal
            df_spat = pd.concat([df_num_state, df_trial_num, csv_spat], axis=1)
            df_spatiotemporal = pd.concat([df_spatiotemporal, df_spat], ignore_index=True)




        df_angles_traj = pd.DataFrame()
        for csv_trajectories in subject.glob('**/*_linear_kinematics.csv'):
            print(csv_trajectories)
                # just like before we save the number of the trial in another dataframe
            trial_str = str(csv_trajectories)
            trial_number = int(trial_str[trial_str.find("_walk_") + 6 : trial_str.find("_linear_kinematics")])
            df_trial_num = pd.Series(trial_number, name='trial_number')

            
        
            #read file and delete three first rows (missing info) and the fourth (units).
            csv_traj = pd.read_excel(csv_trajectories, engine='xlrd')
            csv_traj.drop([0, 1, 2, 3, 4], inplace=True)
        
            #delete time and frame columns
            csv_traj.drop(csv_traj.columns[[0, 1]], axis = 1, inplace = True)
        
            #set first row as dataframe header
            #set all headers to lower letters
            csv_traj = csv_traj[0:]
            csv_traj.columns = trajectories_header
            csv_traj.columns = csv_traj.columns.str.lower()
            csv_traj.reset_index(drop=True, inplace=True)
        
            #calculate range of motion for each joint and set values as a df
            #rom = csv_traj.max() - csv_traj.min()
            #df_rom = pd.DataFrame([rom.values], columns=csv_traj.columns)
        
            # concatenate subject number, state and roms
            #df_trial = pd.concat([df_num_state, df_trial_num, df_rom], axis=1)
            df_trial = pd.concat([df_num_state, df_trial_num, csv_traj], axis=1)
            df_angles_traj = pd.concat([df_angles_traj, df_trial], ignore_index=True)
            

C3Dfiles/SUB22_on/SUB22_on_walk_16_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_3_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_14_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_9_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_1_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_7_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_12_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_5_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_10_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_2_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_15_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_8_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_13_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_6_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_11_linear_kinematics.csv
C3Dfiles/SUB22_on/SUB22_on_walk_4_linear_kinematics.csv
C3Dfiles/SUB16_on/SUB16_on_walk_18_linear_kinematics.csv
C3Dfiles/SUB16_on/SUB16_on_walk_8_linear

In [80]:
df_angles_traj

,subject_number,state,trial_number,com_ap,com_vertical,com_ml,clav_ap,clav_vertical,clav_ml,strn_ap,...,r.mt2_ml,l.knee.medial_ap,l.knee.medial_vertical,l.knee.medial_ml,l.ankle.medial_ap,l.ankle.medial_vertical,l.ankle.medial_ml,l.mt2_ap,l.mt2_vertical,l.mt2_ml
0,1.0,on,12.0,-0.03872,0.93484,-0.29107,0.044464,1.356315,-0.297156,NaN,...,-0.184822,-0.068082,0.493005,-0.277437,-0.193917,0.073784,-0.260435,-0.075878,0.033083,-0.345829
1,NaN,NaN,NaN,-0.03133,0.93424,-0.29079,0.052321,1.355752,-0.296902,NaN,...,-0.185386,-0.06469,0.492743,-0.277112,-0.193337,0.074237,-0.259962,-0.075969,0.033011,-0.34583
2,NaN,NaN,NaN,-0.02395,0.9336,-0.29049,0.060203,1.355112,-0.296619,NaN,...,-0.186206,-0.061244,0.492472,-0.276798,-0.192746,0.074737,-0.259494,-0.076063,0.032947,-0.345829
3,NaN,NaN,NaN,-0.01655,0.93292,-0.29019,0.068109,1.354399,-0.296307,NaN,...,-0.187293,-0.057719,0.492189,-0.276503,-0.192149,0.075286,-0.259044,-0.076158,0.032892,-0.345828
4,NaN,NaN,NaN,-0.00913,0.93218,-0.28988,0.076036,1.353617,-0.295965,NaN,...,-0.188643,-0.054089,0.491887,-0.276234,-0.191549,0.075884,-0.258628,-0.076251,0.032844,-0.345828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,NaN,NaN,NaN,2.56194,0.93804,-0.32065,2.645197,1.369729,-0.320529,2.657614,...,-0.27375,2.669958,0.504739,-0.338093,2.414772,0.157927,-0.346371,2.483142,0.072738,-0.440745
7497,NaN,NaN,NaN,2.56926,0.9385,-0.32099,2.652581,1.370041,-0.320958,2.664769,...,-0.27381,2.681257,0.50696,-0.337924,2.434938,0.153427,-0.348442,2.506431,0.071252,-0.443223
7498,NaN,NaN,NaN,2.57627,0.93906,-0.32125,2.659964,1.370253,-0.321401,2.67192,...,-0.273882,2.692234,0.508981,-0.337602,2.455373,0.148854,-0.350419,2.529949,0.069744,-0.445519
7499,NaN,NaN,NaN,2.58364,0.93936,-0.32158,2.667351,1.370364,-0.321857,2.679072,...,-0.273965,2.703018,0.510957,-0.337416,2.47605,0.14425,-0.352288,2.553685,0.068245,-0.44759


In [81]:
df_trial

,subject_number,state,trial_number,com_ap,com_vertical,com_ml,clav_ap,clav_vertical,clav_ml,strn_ap,...,r.mt2_ml,l.knee.medial_ap,l.knee.medial_vertical,l.knee.medial_ml,l.ankle.medial_ap,l.ankle.medial_vertical,l.ankle.medial_ml,l.mt2_ap,l.mt2_vertical,l.mt2_ml
0,1.0,on,7.0,-0.18365,0.93122,-0.29898,-0.088375,1.354099,-0.297642,NaN,...,-0.186403,-0.255949,0.489076,-0.292737,-0.426995,0.084365,-0.28795,-0.313388,0.033047,-0.374591
1,NaN,NaN,NaN,-0.175,0.93024,-0.29837,-0.079641,1.353008,-0.296836,NaN,...,-0.186717,-0.249661,0.488357,-0.292817,-0.426193,0.085893,-0.288378,-0.313234,0.033,-0.374639
2,NaN,NaN,NaN,-0.16634,0.92921,-0.29773,-0.070902,1.351881,-0.296002,NaN,...,-0.187265,-0.243204,0.487578,-0.292935,-0.42536,0.087507,-0.288882,-0.313062,0.032952,-0.374692
3,NaN,NaN,NaN,-0.15764,0.92814,-0.29709,-0.06216,1.350725,-0.295136,NaN,...,-0.188019,-0.236591,0.486746,-0.293075,-0.424483,0.089209,-0.289448,-0.312873,0.032905,-0.374747
4,NaN,NaN,NaN,-0.1489,0.92704,-0.29643,-0.053417,1.349549,-0.294236,NaN,...,-0.18895,-0.229831,0.485869,-0.293218,-0.423545,0.091004,-0.290061,-0.312671,0.032858,-0.374803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,NaN,NaN,NaN,2.56194,0.93804,-0.32065,2.645197,1.369729,-0.320529,2.657614,...,-0.27375,2.669958,0.504739,-0.338093,2.414772,0.157927,-0.346371,2.483142,0.072738,-0.440745
334,NaN,NaN,NaN,2.56926,0.9385,-0.32099,2.652581,1.370041,-0.320958,2.664769,...,-0.27381,2.681257,0.50696,-0.337924,2.434938,0.153427,-0.348442,2.506431,0.071252,-0.443223
335,NaN,NaN,NaN,2.57627,0.93906,-0.32125,2.659964,1.370253,-0.321401,2.67192,...,-0.273882,2.692234,0.508981,-0.337602,2.455373,0.148854,-0.350419,2.529949,0.069744,-0.445519
336,NaN,NaN,NaN,2.58364,0.93936,-0.32158,2.667351,1.370364,-0.321857,2.679072,...,-0.273965,2.703018,0.510957,-0.337416,2.47605,0.14425,-0.352288,2.553685,0.068245,-0.44759


In [73]:

       # Repeat process for trajectories
for csv_trajectories in subject.glob('**/*_linear_kinematics.csv'):
    print(csv_trajectories)
        # just like before we save the number of the trial in another dataframe
    trial_str = str(csv_trajectories)
    trial_number = int(trial_str[trial_str.find("_walk_") + 6 : trial_str.find("_linear_kinematics")])
    df_trial_num = pd.Series(trial_number, name='trial_number')

    #read file and delete three first rows (missing info) and the fourth (units).
    csv_traj = pd.read_excel(csv_trajectories, engine='xlrd')
    csv_traj.drop([0, 1, 2, 3, 4], inplace=True)

    #delete time and frame columns
    csv_traj.drop(csv_traj.columns[[0, 1]], axis = 1, inplace = True)

    #set first row as dataframe header
    #set all headers to lower letters
    csv_traj = csv_traj[0:]
    csv_traj.columns = trajectories_header
    csv_traj.columns = csv_traj.columns.str.lower()
    csv_traj.reset_index(drop=True, inplace=True)

    #calculate range of motion for each joint and set values as a df
    #rom = csv_traj.max() - csv_traj.min()
    #df_rom = pd.DataFrame([rom.values], columns=csv_traj.columns)

    # concatenate subject number, state and roms
    #df_trial = pd.concat([df_num_state, df_trial_num, df_rom], axis=1)
    df_trial = pd.concat([df_num_state, df_trial_num, csv_traj], axis=1)
    df_angles = df_angles.append(df_trial, ignore_index=True)

In [63]:
df_angles

,subject_number,state,trial_number,trunk_lateral_flexion,trunk_rotation,trunk_flx_extension,left_shoulder_add_abduction,left_shoulder_int_external_rotation,left_shoulder_flx_extension,right_shoulder_add_abduction,...,left_ankle_dorsi_plantarflexion,right_ankle_inv_eversion,right_ankle_add_abduction,right_ankle_dorsi_plantarflexion,left_foot_inv_eversion,left_foot_int_external_rotation,left_foot_df_plantarflexion,right_foot_inv_eversion,right_foot_int_external_rotation,right_foot_df_plantarflexion
0,22,on,15,2.97897,7.90822,3.46405,2.62925,7.44000,3.67538,2.25357,...,28.94538,21.10539,21.09880,26.11941,16.88379,15.08970,67.90324,18.90389,15.11434,70.07257
1,22,on,12,2.95278,6.07138,3.67487,3.08497,5.54828,4.40020,2.24671,...,23.84957,23.18557,22.43639,27.35110,11.67824,12.63517,65.35984,23.52206,15.63506,68.27096
2,22,on,11,2.86238,7.05173,2.70137,2.72333,5.98249,4.07167,2.71655,...,25.56256,20.43074,18.83621,24.30115,16.60968,12.20308,61.42190,17.72578,12.67905,67.35457
3,22,on,16,2.51971,6.33676,4.21162,3.58620,8.06377,6.83211,5.36561,...,26.40672,19.24308,19.30332,25.49302,11.07394,12.02889,68.72693,17.14466,14.70980,69.99992
4,22,on,10,2.13155,6.12791,2.98573,3.33085,7.23728,4.59507,1.71036,...,24.79512,20.42808,18.09145,24.39168,12.56443,12.50767,65.63166,16.15997,12.74538,67.42650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,1,on,6,2.52825,5.40877,4.74835,4.62039,12.47673,14.47797,6.03567,...,20.31854,14.95585,13.17234,27.77969,12.09246,12.49548,59.83803,18.82516,12.72124,67.34195
957,1,on,18,5.14639,6.77762,4.43540,2.62209,6.09964,13.27502,8.07956,...,28.65829,15.96603,17.71778,32.40628,17.51061,11.84632,64.85763,21.83822,13.10276,72.81008
958,1,on,12,2.50659,6.05873,4.60405,4.79872,10.49295,16.56628,4.10215,...,27.67690,15.67198,19.29226,31.59534,17.43477,14.79860,65.90461,17.34572,9.90335,75.18902
959,1,on,1,4.69721,3.80865,4.34763,5.93798,9.41827,19.23258,10.50362,...,20.26136,14.34632,10.05948,27.20666,12.97018,15.44581,60.58130,13.90391,13.48655,74.59862


In [61]:
# Repeat process for trajectories
for csv_trajectories in subject.glob('**/*_linear_kinematics.csv'):
    print(csv_trajectories)
    # just like before we save the number of the trial in another dataframe
    trial_str = str(csv_trajectories)
    trial_number = int(trial_str[trial_str.find("_walk_") + 6 : trial_str.find("_linear_kinematics")])
    df_trial_num = pd.Series(trial_number, name='trial_number')

    if "SUB01" in csv_trajectories.name:
        # Read each CSV file into a DataFrame
        df = pd.read_csv(csv_trajectories)
        
        # Append the DataFrame to the 'forPrint' DataFrame
        forPrint = forPrint.append(df, ignore_index=True)


    # read file and delete three first rows (missing info) and the fourth (units).
    csv_traj = pd.read_excel(csv_trajectories, engine='xlrd')
    csv_traj.drop([0, 1, 2, 3, 4], inplace=True)

    # delete time and frame columns
    csv_traj.drop(csv_traj.columns[[0, 1]], axis=1, inplace=True)

    # set first row as dataframe header
    # set all headers to lowercase
    csv_traj = csv_traj[0:]
    csv_traj.columns = trajectories_header
    csv_traj.columns = csv_traj.columns.str.lower()
    csv_traj.reset_index(drop=True, inplace=True)

    # append entire trajectory data without calculating range of motion
    df_trial = pd.concat([df_num_state, df_trial_num, csv_traj], axis=1)
    df_angles = df_angles.append(df_trial, ignore_index=True)
